In [1]:
using Mamba

In [2]:
using Stan

In [3]:
const bernoullistanmodel = "
data { 
  int<lower=0> N; 
  int<lower=0,upper=1> y[N];
} 
parameters {
  real<lower=0,upper=1> theta;
} 
model {
  theta ~ beta(1,1);
    y ~ bernoulli(theta);
}
";

In [4]:
stanmodel = Stanmodel(name="bernoulli", model=bernoullistanmodel)

  name =                    "bernoulli"
  nchains =                 4
  num_samples =             1000
  num_warmup =                   1000
  thin =                    1
  useMamba =                true
  mambaThinning =           1
  monitors =                String[]
  model_file =              "bernoulli.stan"
  data_file =               ""
  output =                  Output()
    file =                    ""
    diagnostics_file =        ""
    refresh =                 100
  method =                  Sample()
    num_samples =             1000
    num_warmup =              1000
    save_warmup =             false
    thin =                    1
    algorithm =               HMC()
      engine =                  NUTS()
        max_depth =               10
      metric =                  

Stan.diag_e
      stepsize =                1.0
      stepsize_jitter =         1.0
    adapt =                   Adapt()
      gamma =                   0.05
      delta =                   0.8
      kappa =                   0.75
      t0 =                      10.0
      init_buffer =             75
      term_buffer =             50
      window =                  25


In [5]:
bernoullidata = Dict("N" => 10, "y" => [0, 1, 0, 1, 0, 0, 0, 0, 0, 1])

Dict{String,Any} with 2 entries:
  "N" => 10
  "y" => [0, 1, 0, 1, 0, 0, 0, 0, 0, 1]

### Sometimes output of CmdStan is not displayed properly. Usually re-running this cell by itself fixes this.

### Also note that if the stan script has already been compiled and built, make will not repeat these steps.

In [6]:
rc, sim1 = stan(stanmodel, [bernoullidata], pwd(), CmdStanDir=CMDSTAN_HOME);


make: `/Users/rob/Downloads/tmp/bernoulli' is up to date.

Length of data array is not equal to nchains,
all chains will use the first data dictionary.

Calling /Users/rob/Projects/StanSupport/cmdstan/bin/stansummary to infer across chains.

Inference for Stan model: bernoulli_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.0092, 0.016, 0.016, 0.016) seconds, 0.057 seconds total
Sampling took (0.022, 0.040, 0.043, 0.040) seconds, 0.14 seconds total

                Mean     MCSE  StdDev    5%   50%   95%  N_Eff  N_Eff/s    R_hat
lp__            -8.1  1.6e-02    0.67  -9.4  -7.9  -7.6   1857    12828  1.0e+00
accept_stat__   0.92  1.9e-03    0.12  0.65  0.97   1.0   4000    27630  1.0e+00
stepsize__       1.0  8.9e-02    0.13  0.95  1.00   1.3    2.0       14  5.6e+13
treedepth__      1.4  7.8e-03    0.49   1.0   1.0   2.0   3883    26820  1.0e+00
n_leapfrog__     2.5  5.0e-02     1.3   1.0   3.0   3.0    65

### What's the structure of a Mamba.Chains object

In [10]:
typeof(sim1)

Mamba.Chains

In [7]:
size(sim1)

(1000, 8, 4)

In [8]:
fieldnames(sim1)

4-element Array{Symbol,1}:
 :value 
 :range 
 :names 
 :chains

In [9]:
sim1.names

8-element Array{AbstractString,1}:
 "lp__"         
 "accept_stat__"
 "stepsize__"   
 "treedepth__"  
 "n_leapfrog__" 
 "divergent__"  
 "energy__"     
 "theta"        

### If return code == 0, filter sim1 output and call Mamba's describe().

In [11]:
if rc == 0
  println("Subset Sampler Output")
  sim = sim1[1:1000, ["lp__", "theta", "accept_stat__"], :]
  describe(sim)
end

Subset Sampler Output
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                  Mean        SD       Naive SE       MCSE      ESS
         lp__ -8.12864507 0.67144367 0.0106164566 0.0154961591 1000
        theta  0.32867301 0.12640946 0.0019987091 0.0031223402 1000
accept_stat__  0.91868808 0.12243779 0.0019359115 0.0029870001 1000

Quantiles:
                 2.5%       25.0%       50.0%      75.0%       97.5%  
         lp__ -9.9433640 -8.29392750 -7.8834950 -7.69665500 -7.6387900
        theta  0.1188135  0.23013675  0.3193140  0.41652875  0.5894945
accept_stat__  0.5466045  0.89072425  0.9702025  1.00000000  1.0000000



### Brooks, Gelman and Rubin Convergence Diagnostic

### Note that all below diagnostic routines should be bracketed by `if rc == 0 ... end`.

In [14]:
try
  gelmandiag(sim, mpsrf=true, transform=true) |> display
catch e
  #println(e)
  gelmandiag(sim, mpsrf=false, transform=true) |> display
end

               PSRF 97.5%
         lp__ 1.003 1.006
        theta 1.004 1.012
accept_stat__ 1.052 1.088
 Multivariate 1.032   NaN




Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] log(::Array{Float64,2}) at ./deprecated.jl:57
 [3] link(::Mamba.Chains) at /Users/rob/.julia/v0.6/Mamba/src/output/chains.jl:242
 [4] #gelmandiag#104(::Float64, ::Bool, ::Bool, ::Function, ::Mamba.Chains) at /Users/rob/.julia/v0.6/Mamba/src/output/gelmandiag.jl:9
 [5] (::Mamba.#kw##gelmandiag)(::Array{Any,1}, ::Mamba.#gelmandiag, ::Mamba.Chains) at ./<missing>:0
 [6] include_string(::String, ::String) at ./loading.jl:515
 [7] include_string(::Module, ::String, ::String) at /Users/rob/.julia/v0.6/Compat/src/Compat.jl:407
 [8] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/rob/.julia/v0.6/IJulia/src/execute_request.jl:154
 [9] eventloop(::ZMQ.Socket) at /Users/rob/.julia/v0.6/IJulia/src/eventloop.jl:8
 [10] (::IJulia.##14#17)() at ./task.jl:335
while loading In[14], in expression starting on line 2


### Geweke Convergence Diagnostic

In [15]:
gewekediag(sim) |> display

              Z-score p-value
         lp__   0.657  0.5113
        theta   0.994  0.3203
accept_stat__   2.438  0.0148

              Z-score p-value
         lp__  -1.825  0.0679
        theta   0.000  0.9998
accept_stat__   0.018  0.9854

              Z-score p-value
         lp__  -0.281  0.7783
        theta  -0.034  0.9731
accept_stat__  -0.982  0.3260

              Z-score p-value
         lp__  -0.988  0.3232
        theta  -1.877  0.0606
accept_stat__  -0.725  0.4682



### Highest Posterior Density Intervals

In [16]:
hpd(sim) |> display

              95% Lower 95% Upper
         lp__ -9.429950 -7.638170
        theta  0.115025  0.581819
accept_stat__  0.654388  1.000000



### Cross-Correlations

In [17]:
cor(sim) |> display

                 lp__       theta    accept_stat__
         lp__ 1.00000000  0.04377643    0.41730895
        theta 0.04377643  1.00000000   -0.07680032
accept_stat__ 0.41730895 -0.07680032    1.00000000



### Lag-Autocorrelations

In [18]:
autocor(sim) |> display

                  Lag 1        Lag 5         Lag 10         Lag 50   
         lp__   0.32016258  -0.017256870    0.027960976  0.0324091917
        theta   0.38801658   0.012344096   -0.034038246  0.0097314316
accept_stat__  -0.09665741   0.026337036    0.041742658 -0.0422834258

                  Lag 1        Lag 5         Lag 10         Lag 50   
         lp__  0.276833585 -0.0011036154    0.071220839  0.0066471642
        theta  0.402378185  0.0397211474    0.008000695 -0.0314988447
accept_stat__ -0.046943386 -0.0533491235    0.050268445 -0.0040709876

                  Lag 1        Lag 5         Lag 10         Lag 50   
         lp__   0.42141370  0.0199876342    0.065692544 -0.0046114129
        theta   0.42027822 -0.0060865494    0.014326021 -0.0042476505
accept_stat__  -0.07515192 -0.0131331858    0.057966881  0.0656034097

                  Lag 1        Lag 5         Lag 10         Lag 50   
         lp__  0.351582623    0.02746748  0.00041632276   0.036462336
        theta  0.

### Create plot files

In [19]:
p = plot(sim, [:trace, :mean, :density, :autocor], legend=true);
draw(p, ncol=4, filename="summaryplot", fmt=:svg)
draw(p, ncol=4, filename="summaryplot", fmt=:pdf)

  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely near In[19]:2
  likely ne

#### @__FILE__ only returns a useful value if the julia script is loaded from a file.

In [20]:
ProjDir=@__FILE__

""

In [21]:
pwd()

"/Users/rob/Downloads"

In [22]:
ProjDir=pwd()

"/Users/rob/Downloads"

In [23]:
Pkg.dir("Stan")

"/Users/rob/.julia/v0.6/Stan"

In [24]:
VERSION

v"0.6.0"

In [25]:
is_apple()

true